In [1]:
''' imports '''

# set auto reload imported modules
%load_ext autoreload
%autoreload 2

# numpy for array handling
import numpy as np

# import pytorch core libs
import torch


# midi file import and parse
from mido import Message, MidiFile, MidiTrack, MetaMessage

import pickle

# audio playback widget
import IPython.display as ipd

%matplotlib widget
import matplotlib.pyplot as plt


# add sample-rnn libs directory to path
import sys
sys.path.append('../melodyrnn/')

from dataset import MelodyDataset

from utils import build_audio


In [2]:
''' load some generated samples '''

with open('../data/melody/smpls-new-trans-02', 'rb') as file:
    samples = pickle.load(file)
    

In [3]:
samples[0].shape

(15000,)

In [4]:
''' convert output to note matrix '''

m = samples[1000]

_pad = 32
M = np.zeros((128, m.shape[0]+_pad))
for i in range(m.shape[0]):
    if m[i] != 0:
        M[int(m[i]), _pad+i] = 1

''' display melody note matrix '''
fig = plt.figure(figsize=(7,3))
ax = fig.add_subplot(111)

# set downsample time axis [int]
ds = 64

# display track note matrix
plt.imshow(M[::-1,::ds], cmap = 'bone_r', alpha = 1.)

# format figure and display
#ax.set_ylim(35, 95)

ax.set_ylabel('midi note')
ax.set_xlabel('time [smpl]')

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
''' build and play audio sample '''

# set sample rate
sr = 16000

# set frame rate of midi track, adjust for playback speed
fr = (2**18 * 10) // 2


# generate audio each track
#audio = build_audio(M[:, ds*_:ds*__], sr = sr, fr = fr)
audio = build_audio(M[:, :], sr = sr, fr = fr, soft = 1e-4)

# normalise audio
audio /= audio.max()

# display playback widget
ipd.Audio(audio, rate = sr)


In [6]:
''' segment melody output '''

smpl = samples[10]


In [7]:
plt.figure(figsize = (9,3))

plt.plot(smpl, '.')

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
''' get seg splits by note delta / zero gap '''

# note jump for split
delta = 12

# set length
_len = 500
_max = 800

# min/max notes
#_notes = (4, 20)
_notes = 0.7

_range = 20

# set zero padding
_pad = 16

segs = []


# iterate over samples
for smpl in samples[:5:]:

    # get index of start gap in sample, zero start, full end
    j = [0] + [ i for i in range(smpl.shape[0]-1) if abs(smpl[i+1] - smpl[i]+1) > delta ] + [smpl.shape[0]]

    # iterate over segment splits
    for i in range(len(j)-1):

        # get segment from splits
        seg = smpl[j[i]:j[i+1]].astype(np.int16)

        # ensure not segment of zeros, more than 3 notes
        if (list(set(seg)) != [0]) and (len(set(seg)) > 3):
        
            # strip leading, trailing zeros
            z = np.where(seg != 0)[0]
            seg = seg[z[0]:z[::-1][0]]

            # filter empty segs
            if seg.shape[0] != 0:
                
                # filter for minimum length; max note range
                if (seg.shape[0] > _len) and np.ptp( seg[np.where(seg != 0)] ) < _range:

                    # count notes in segment
                    notes = len([ i for i in range(_len-1) if seg[i+1] != seg[i] ])

                    # filter for minimum notes per length
                    #if (notes >= _notes[0]) and (notes <= _notes[1]):
                    if (notes >= (_notes * seg.shape[0] // 100)):
                        
                        if seg.shape[0] > _max:
                        
                            seg = seg[:_max]
                        
                        # pad std len zeros
                        seg = np.pad(seg, (_pad))

                        #print(seg.shape[0], notes)

                        # store segment
                        segs.append(seg)
    
print('\t', len(segs))

	 2


In [9]:
''' check seg and filter '''

plt.figure(figsize = (9,3))

for i, seg in enumerate(segs):
    plt.plot(seg[16:-16], '.', label = i, alpha = 0.3)

plt.legend()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
''' convert output to note matrix '''

m = segs[0]

_pad = 0
M = np.zeros((128, m.shape[0]+_pad))
for i in range(m.shape[0]):
    if m[i] != 0:
        M[int(m[i]), _pad+i] = 1

''' display melody note matrix '''
fig = plt.figure(figsize=(7,3))
ax = fig.add_subplot(111)

# set downsample time axis [int]
ds = 10

# display track note matrix
plt.imshow(M[::,::ds], cmap = 'bone_r', alpha = 1.)

# format figure and display
ax.set_ylim(35, 95)

ax.set_ylabel('midi note')
ax.set_xlabel('time [smpl]')

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
''' build and play audio sample '''

# set sample rate
sr = 16000

# set frame rate of midi track, adjust for playback speed
fr = (2**18 * 10) // 1


# generate audio each track
#audio = build_audio(M[:, ds*_:ds*__], sr = sr, fr = fr)
audio = build_audio(M[:, :], sr = sr, fr = fr, soft = 1e-4)

# normalise audio
audio /= audio.max()

# display playback widget
ipd.Audio(audio, rate = sr)


In [13]:
''' get notes from melody sample '''

def melody2notes(melody):
    
    # initialise note event list
    events = []

    # iterate each timestep
    for dt in range(M.shape[1])[:-1]:

        # get note index for note onset at timestep
        j = np.where(M[:, dt+1] > M[:, dt])[0]

        # note onset at timestep
        if len(j) != 0:

            # iterate each note onset
            for k in j:

                # store note index, onset time
                events.append( [k, dt] )


    # initialise note list
    notes = []

    # iterate over note events
    for note in events[:]:

        # get time from onset to note end (zero velocity)
        dt = np.where( M[note[0], note[1]+1:] == 0 )[0]

        # ensure duration exists
        if len(dt) != 0:

            # store note as list [note, onset, duration]
            notes.append([*note, dt[0]])

    return notes


In [14]:
''' convert, export output samples to midi files '''

mod = 8
ds = 10

for j in range(len(segs))[:]:
    m = segs[j]

    M = np.zeros((128, m.shape[0]))
    for i in range(m.shape[0]-1):
        if m[i] != 0:
            M[int(m[i]), _pad+i] = 1

    # get notes from Matrix
    notes = melody2notes(M)
    
    #print(len(notes))

    # init midi file, track
    mid = MidiFile()

    track = MidiTrack()
    mid.tracks.append(track)


    # set end track time
    #track.append(MetaMessage('end_of_track', time = notes[-1][1] + notes[-1][2] ))

    #<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
    track.append(MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))

    track = MidiTrack()
    mid.tracks.append(track)

    # store midi meta
    #track.append(Message('program_change', program=12, time=0))


    # init time, delta
    time = 0
    delta = 0

    # iterate notes
    for note in notes:

        # unpack note data
        note_number = note[0]
        start = ds * note[1] // mod
        duration = ds * note[2] // mod

        # get time delta, update current time
        if start > time:
            delta = start - time
            time += delta
        else:
            delta = 0

        # insert note start
        track.append(Message('note_on', note = note_number, velocity = 64, time = delta))

        # assume no note overlap
        delta += duration

        # insert note end
        track.append(Message('note_off', note = note_number, velocity = 0, time = delta))


    # save midi file
    #mid.save('../data/melody/midi/var-len-out-{}.midi'.format(j))
    

In [15]:
notes

[[64, 15, 198],
 [66, 213, 187],
 [64, 400, 89],
 [65, 489, 13],
 [64, 502, 64],
 [62, 566, 49]]